In [21]:
import pandas as pd

In [22]:
df1 = pd.read_csv("../Data/eth_exchange_netflow_usd.csv")
df2 = pd.read_csv("../Data/exchange_deposite_withdraw_tx.csv")
df3 = pd.read_csv("../Data/eth_ohlcv_fees_tx_size_fail_rate.csv")
df4 = pd.read_csv("../Data/staking_inflow_withdrawal_total.csv")
df5 = pd.read_csv("../Data/eth_active_addresses.csv")

In [23]:
for df in [df1, df2, df3, df4, df5]:
    df['datetime'] = pd.to_datetime(df['datetime']).dt.tz_localize(None)

merged = (
    df2
    .merge(df1, on="datetime", how="left", suffixes=("", "_exch"))
    .merge(df3, on="datetime", how="left", suffixes=("", "_ohlcv"))
    .merge(df4, on="datetime", how="left", suffixes=("", "_staking"))
    .merge(df5, on="datetime", how="left", suffixes=("", "_addr"))
    .sort_values("datetime")
)

merged = merged.fillna(0)

print(merged.shape)
print(merged.head())

(492480, 17)
             datetime  exchange_depositing_count  exchange_withdrawing_count  \
0 2024-11-05 00:00:00                         63                         112   
1 2024-11-05 00:01:00                         26                          79   
2 2024-11-05 00:02:00                         62                         103   
3 2024-11-05 00:03:00                         15                          93   
4 2024-11-05 00:04:00                         35                          94   

   exchange_netflow_usd  onchain_volume_usd  avg_tx_size_usd  avg_gas_fee_usd  \
0         -7.199263e+04        3.935351e+05       409.080112         1.417779   
1         -1.124779e+04        3.715324e+05       525.505481         1.772062   
2          1.238843e+05        7.649811e+05       827.901572         1.249843   
3         -4.605539e+04        3.820484e+05       633.579355         1.410977   
4          2.130716e+06        7.583283e+06      8636.996496         1.585209   

   avg_priority_fee

In [24]:
merged.to_csv("../Data/dune_eth.csv", index=False)